# Trajectory planning of self driving cars with a continuous method
I implement a simple version of the trajectory planning method described in [Trajectory Planning for BERTHA - a Local, Continuous Method](https://pdfs.semanticscholar.org/bdca/7fe83f8444bb4e75402a417053519758d36b.pdf). To make things easier, I only focus on the longitudinal control. This means that I only plan the velocity decomposition of the car. The mathematical basics behind the method are described below. 

The target of this method is minimizing the cost of a trajectory, given by a cost functional with respect to constraints and thus finding an optimal trajectory. 

# Cost-functional
The desired driving behaviour is described in the functional

$$ J(x(t)) = \frac{1}{2} \int_{t}^{t+T} w_{v}(v(t) - v_{des})^{2} + w_{a}a(t)^{2} + w_{j}j(t)^{2} dt. $$

$v(t)$, $a(t)$ and $j(t)$ describe the velocity, acceleration and jerk of the car. They are related through the planned path coordinate $s(t)$:

$$ v(t) = \dot{s}(t) = \frac{d s}{d t}, $$
$$ a(t) = \ddot{s}(t) =\frac{d v}{d t}, $$
$$ j(t) = \dddot{s}(t) = \frac{d a}{d t}. $$

$v_{des}$ is the desired velocity and $T$ is the optimization timespan. $w_{v}$, $w_{v}$ and $w_{v}$ are weights to manually emphasize the importance of each term in respect to the other terms inside the cost functional. 

# Constraints

## Physical constraints
A vehicle has a certain velocity and acceleration limit,

$$ v(t) \le v_{max}, $$
$$ a(t) \le a_{max}. $$

## Obstacle constraints
The only obstacles I focus on here are other vehicles, that intersect the self driving cars path. The figure below shows an example. The green vehicle is the self driving car. It's path $s(t)$ is intersected by the red cars path. This is why the red car is a dynamic obstacle. 
![](img/small.png)

In the path-time-diagram of the self driving green car the red car is depicted by the blue box. Here the optimization timespan $T$ is equal to 10 seconds. 

![](img/raumzeit.png)

The equations describing the red vehicle are
$$ s(t) \le s_{a}, \quad \in [t_{a}, t_{b}], $$
$$ s(t) \ge s_{b}, \quad \in [t_{a}, t_{b}]. $$
It basically describes that a certain part of green cars path (the intersection) is blocked during a certain time. 

This formulation makes the optimization non-convex. This leads to some problems. One of them is that we obtain several (in this case two) local minima. The diagram below shows that the trajectories converge in different minima (right diagram), depending on the initializations (left picture). To find the global minimum the costs of the two trajectories have to be compared. 

![](img/initial.png)

One way to make the optimization independent of the initialization is to strech the boarders of the obstacle to plus or minus infinity, as shown in the diagrams below. This leads to two convex spaces. 

![](img/konvex.png)

So to find all local minima of the described problem, two optimizations and a comparison of costs is required. However, if an amount of $N$ obstacles is found in the path-time-diagram $2^{N}$ are neccessary. 